# Scrape betting lines from Fanduel

This notebook will be later adapted into a script and integrated into Geiger's Excel spreadsheet for NFL picks. Ideally, it will be as intuitive as Mike's idea of "press a button and it updates lines".

First, we explore what Fanduel site looks like in `bs4`.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)

## What info do we need for the spreadsheet?

https://www.espn.com/nfl/lines

- The two numbers listed under "LINE".
    - Take greater of two numbers, attach it to "TotalPts", other is "Spread"
    - Also pull team names attached to greater #

In [2]:
url = 'https://www.espn.com/nfl/lines'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
parsed = soup.find_all('table')
parsed[0]

<table class="Table Table--align-right" style="border-collapse:collapse;border-spacing:0"><thead class="Table__header-group Table__THEAD"><tr class="Table__sub-header Table__TR Table__even"><th class="Table__TH" title="">1:00 PM</th><th class="Table__TH" title="">REC (ATS)</th><th class="Table__TH" title="">LINE</th><th class="Table__TH" title="">OPEN</th><th class="Table__TH" title="">ML</th><th class="Table__TH" title="">FPI</th></tr></thead><tbody class="Table__TBODY"><tr class="Table__TR Table__TR--sm Table__even" data-idx="0"><td class="Table__TD"><div class="flex items-start mr7"><span class="pr3 TeamLink__Logo"><a class="AnchorLink" href="/nfl/team/_/name/phi/philadelphia-eagles" tabindex="0"></a></span><a class="AnchorLink" data-clubhouse-uid="s:20~l:28~t:21" href="/nfl/team/_/name/phi/philadelphia-eagles" tabindex="0">Philadelphia Eagles</a></div></td><td class="Table__TD">0-0 (0-0-0)</td><td class="Table__TD">48.5</td><td class="Table__TD">48.5</td><td class="Table__TD">+155<

In [3]:
home = []
away = []
total = []
spread = []

for game in parsed:
    try:
        # find team names
        home_name = game.find_all('td')[0].img.get('title')
        away_name = game.find_all('td')[6].img.get('title')
        # get total pts and/or spread
        # the order is inconsistent here
        # line is always relative to favored team
        totalpts = float(game.find_all('td')[2].get_text())
        line = float(game.find_all('td')[8].get_text())
        home.append(home_name)
        away.append(away_name)
        total.append(totalpts)
        spread.append(line)
    except:
        pass

week = pd.DataFrame(list(zip(home, away, total, spread)),
                    columns=['Home', 'Away', 'Total', 'Spread'])
week

,Home,Away,Total,Spread
0,Philadelphia Eagles,Atlanta Falcons,48.5,-3.5
1,Pittsburgh Steelers,Buffalo Bills,48.0,-6.5
2,Minnesota Vikings,Cincinnati Bengals,-3.0,47.0
3,San Francisco 49ers,Detroit Lions,-8.5,46.0
4,Arizona Cardinals,Tennessee Titans,53.5,-3.0
5,Seattle Seahawks,Indianapolis Colts,-3.0,49.0
6,Los Angeles Chargers,Washington,44.5,-1.0
7,New York Jets,Carolina Panthers,45.0,-4.0
8,Jacksonville Jaguars,Houston Texans,-3.0,45.5
9,Cleveland Browns,Kansas City Chiefs,54.5,-5.5


In [4]:
to_switch = week.loc[(week['Total'] < 0)].copy()
to_switch['TempTotal'] = to_switch['Total']
to_switch

,Home,Away,Total,Spread,TempTotal
2,Minnesota Vikings,Cincinnati Bengals,-3.0,47.0,-3.0
3,San Francisco 49ers,Detroit Lions,-8.5,46.0,-8.5
5,Seattle Seahawks,Indianapolis Colts,-3.0,49.0,-3.0
8,Jacksonville Jaguars,Houston Texans,-3.0,45.5,-3.0
11,Green Bay Packers,New Orleans Saints,-3.5,49.5,-3.5
12,Denver Broncos,New York Giants,-3.0,41.5,-3.0
14,Baltimore Ravens,Las Vegas Raiders,-4.0,50.5,-4.0


In [5]:
to_switch['Total'] = to_switch['Spread']
to_switch['Spread'] = abs(to_switch['TempTotal'])
to_switch

,Home,Away,Total,Spread,TempTotal
2,Minnesota Vikings,Cincinnati Bengals,47.0,3.0,-3.0
3,San Francisco 49ers,Detroit Lions,46.0,8.5,-8.5
5,Seattle Seahawks,Indianapolis Colts,49.0,3.0,-3.0
8,Jacksonville Jaguars,Houston Texans,45.5,3.0,-3.0
11,Green Bay Packers,New Orleans Saints,49.5,3.5,-3.5
12,Denver Broncos,New York Giants,41.5,3.0,-3.0
14,Baltimore Ravens,Las Vegas Raiders,50.5,4.0,-4.0


In [6]:
week.update(to_switch)

In [7]:
week

,Home,Away,Total,Spread
0,Philadelphia Eagles,Atlanta Falcons,48.5,-3.5
1,Pittsburgh Steelers,Buffalo Bills,48.0,-6.5
2,Minnesota Vikings,Cincinnati Bengals,47.0,3.0
3,San Francisco 49ers,Detroit Lions,46.0,8.5
4,Arizona Cardinals,Tennessee Titans,53.5,-3.0
5,Seattle Seahawks,Indianapolis Colts,49.0,3.0
6,Los Angeles Chargers,Washington,44.5,-1.0
7,New York Jets,Carolina Panthers,45.0,-4.0
8,Jacksonville Jaguars,Houston Texans,45.5,3.0
9,Cleveland Browns,Kansas City Chiefs,54.5,-5.5


In [8]:
nfl_abbr = pd.read_csv('nfl_teams.csv')
nfl_abbr = nfl_abbr.drop(['ID', 'Conference', 'Division'], axis=1)

week_abbr_temp = week.merge(nfl_abbr, left_on='Home', right_on='Name')
week_abbr_temp = week_abbr_temp.rename(columns={'Abbreviation':'Home Abbrv'})

week_abbr = week_abbr_temp.merge(nfl_abbr, left_on='Away', right_on='Name')
week_final = week_abbr.drop(['Name_x','Name_y', 'Home', 'Away'], axis=1)
week_final = week_final.rename(columns={'Home Abbrv':'Home','Abbreviation':'Away'})

In [9]:
week_final = week_final[['Home','Away','Total','Spread']]
week_final

,Home,Away,Total,Spread
0,PHI,ATL,48.5,-3.5
1,PIT,BUF,48.0,-6.5
2,MIN,CIN,47.0,3.0
3,SF,DET,46.0,8.5
4,ARI,TEN,53.5,-3.0
5,SEA,IND,49.0,3.0
6,JAX,HOU,45.5,3.0
7,CLE,KC,54.5,-5.5
8,MIA,NE,43.5,-3.0
9,GB,NO,49.5,3.5


# Hell yeah!

We got it and in a cool and easy to digest and digestible and easy to digest format! Let's turn that into a script later. For now, let's do things `gsheets` package and figure out how to write this to a gsheet.